In [59]:
# Before running the given file we need to mount colab to our drive
# After that we need to create a folder and we need to add both our data set files

#importlibraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import string
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_hub as hub
from sklearn.metrics import confusion_matrix

In [2]:
from google.colab import drive
drive.mount ('/gdrive')

Mounted at /gdrive


In [3]:
os.chdir('/gdrive/My Drive/')

In [4]:
data_train=pd.read_csv('CovidProject/Corona_NLP_train.csv',encoding='latin1')
data_test=pd.read_csv('CovidProject/Corona_NLP_test.csv',encoding='latin1')
data_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [5]:
data_test.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [6]:
print(data_train['Sentiment'].unique())

['Neutral' 'Positive' 'Extremely Negative' 'Negative' 'Extremely Positive']


In [7]:
# Merging the extreme cases with their parent class
for i in range(0,len(data_train)):
    if(data_train['Sentiment'][i]=='Extremely Negative'):
        data_train['Sentiment'][i]='Negative'
    elif(data_train['Sentiment'][i]=='Extremely Positive'):
        data_train['Sentiment'][i]='Positive'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
print(data_train['Sentiment'].unique())

['Neutral' 'Positive' 'Negative']


In [9]:
data_train['Sentiment'].isna().sum()


0

In [10]:
# Converting labels to numericals
le=LabelEncoder()

In [11]:
data_train['Sentiment']=le.fit_transform(data_train['Sentiment'])

In [12]:
y=pd.get_dummies(data_train['Sentiment'])

In [13]:
data_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,2
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,2
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,2
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0


In [14]:
print(le.inverse_transform([0,1,2]))

['Negative' 'Neutral' 'Positive']


In [15]:
n=0
ne=0
p=0

In [16]:
for i in range(0,len(data_train)):
    if(data_train['Sentiment'][i]==0):
        n=n+1
    elif(data_train['Sentiment'][i]==1):
        ne=ne+1
    elif(data_train['Sentiment'][i]==2):
        p=p+1

In [17]:
# Total data for each class
print('Total Labels belonging to Negative Category are ',n)
print('Total Labels belonging to Neutral Category are ',ne)
print('Total Labels belonging to Positive Category are ',p)

Total Labels belonging to Negative Category are  15398
Total Labels belonging to Neutral Category are  7713
Total Labels belonging to Positive Category are  18046


In [18]:
print('Total entries in the train dataset is ',len(data_train))

Total entries in the train dataset is  41157


In [19]:
##Segregating the tweet class from their original dataset
X=data_train.iloc[:,4]

In [20]:

train_text,validation_text,train_labels,validation_labels=train_test_split(X.to_numpy(),y.to_numpy(),test_size=0.2,random_state=42)

In [21]:
##printing validation labels to check if they are in one hot encoded form or not
validation_labels

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0]], dtype=uint8)

In [22]:
type(train_text)

numpy.ndarray

In [23]:
# Cleaning textual data(Removing Hashtags,Mentions,Links,etc)
for i in range(0,len(train_text)):
    temp=train_text[i]
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
    train_text[i]=temp
    ##print(temp)

In [24]:
for i in range(0,len(validation_text)):
    temp=validation_text[i]
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
    validation_text[i]=temp
    ##print(temp)

In [25]:
# Print some random strings to check if the data has been preprocessed and cleaned well
for i in range(0,10):
    print('New Tweet')
    print('***********')
    random_number=random.randint(0,len(train_text)-1)
    print(train_text[random_number])

New Tweet
***********
 I know is out for ratings and billionaires just look at how much made when he drove down the price of stocks just to buy them at lower prices But letÂs speak facts 
New Tweet
***********
Strangers stepping in to help one another during COVID pandemic Strangers stepping in to help one another during COVID pandemic Lauren Verno Consumer investigative reporter Published March 
New Tweet
***********
PLEASE STAY HOME or not STAY HOME Pharamacy yes Grocery store yes Cocktails with friends NO Basketball games NO STAY HOME Please
New Tweet
***********
As of this morning we had price gouging complaints to our consumer unit Agents with my office are in the process of following up on every one of them Watch my interview w on what to do if you suspect price gouging 
New Tweet
***********
How cheap consumer products in this case a decathlon snorkelling mask and some lateral thinking can help in the fight against 
New Tweet
***********
Finally made it into our local supermark

In [26]:
for i in range(0,10):
    print('New Tweet')
    print('***********')
    random_number=random.randint(0,len(validation_text)-1)
    print(validation_text[random_number])

New Tweet
***********
Why we stock up on water cause utility companies will shut you off in the middle of a pandemic the schools close thier doors you lose out on work cause your kid has no where to go and you canÂt afford months worth of food 
New Tweet
***********
Are Gift Cards the War Bonds of the COVIDÂ Era Maybe So but Issuers Still Need to Consider the Consumer Protection Laws 
New Tweet
***********
Covid is changing the world as we know it The ability to adapt a Digital Marketing is crucial to create meaningful content and Content such as recipes DIY projects home workouts livestreams online shopping and e learning are some ideas 
New Tweet
***********
 In China people must use the similar App to scan a QR code before taking any public transit shopping in mall or supermarket entering any office building You can be traceable if they found a covid cases 
New Tweet
***********
Will the epidemic harm UK 
New Tweet
***********
 ÂCOVID  Â gt 
New Tweet
***********
I hope this ma

In [27]:
type(train_text)

numpy.ndarray

In [28]:
sum_length_of_tweet=0

In [29]:
##finding the average length of each tweet
for i in X:
    temp=i
    sum_length_of_tweet=sum_length_of_tweet+len(temp.split())

In [30]:
max_length=round(sum_length_of_tweet/len(X))

In [31]:
print(max_length)

31


In [32]:
# Tokenization and Padding the textual data
Tokenizer?

In [33]:
tokenizer=Tokenizer(num_words=10000,oov_token='</OOV>')

In [34]:
len(tokenizer.word_index)

0

In [35]:
tokenizer.fit_on_texts(train_text)

In [36]:
train_text_sequences=tokenizer.texts_to_sequences(train_text)

In [37]:
train_text_padded=pad_sequences(train_text_sequences,maxlen=max_length,padding='post')

In [38]:
validation_text_sequences=tokenizer.texts_to_sequences(validation_text)

In [39]:
validation_text_padded=pad_sequences(validation_text_sequences,maxlen=max_length,padding='post')

In [40]:
# Printings some random examples to check the function
for i in range(0,10):
    index=random.randint(0,len(train_text)-1)
    print('Original Sentence')
    print('*******')
    print(train_text[index])
    print('Padded Sentence')
    print('*********')
    print(train_text_padded[index])

Original Sentence
*******
Coronavirus concerns drag consumer confidence in Africa 
Padded Sentence
*********
[  79  707 5243   36  713    6 1575    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
Original Sentence
*******
i never realized how much my face itched at the grocery store before the 
Padded Sentence
*********
[  12  294 2963   51  171   37  196    1   16    2   26   18  176    2
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
Original Sentence
*******
Headed to the grocery store wish me luck Aliso Viejo California 
Padded Sentence
*********
[3251    3    2   26   18 1026   86 2625    1    1 1016    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
Original Sentence
*******
Out of stores of are closed amp rest selling only essential items That is one of the reasons the stock is in lower circuit today Supermarket chai

In [41]:
for i in range(0,10):
    index=random.randint(0,len(validation_text)-1)
    print('Original Sentence')
    print('*******')
    print(validation_text[index])
    print('Padded Sentence')
    print('*********')
    print(validation_text_padded[index])

Original Sentence
*******
Positives from the Pandemic Rediscovering local butchers and grocers No crowds no panic buyers plenty of good quality food on offer and the real sense of supporting smaller businesses at this time 
Padded Sentence
*********
[  30    2   62    1  106 4372    4 1993   56 1533   56   65  671  686
    5  158 1289   19   14  673    4    2  319  969    5 1136 2504  223
   16   15   66]
Original Sentence
*******
 Amazing healthcare staff coming up with ways to educate people 
Padded Sentence
*********
[ 870  388  138  343   41   21  632    3 4563   29    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
Original Sentence
*******
Ask about our new policies during these sensitive times DM for prices and availability 
Padded Sentence
*********
[ 606   59   40   91 1976   53  101 4831  182 1397    8   17    4 1750
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
Original Sentenc

In [42]:
train_text_padded.shape[1]

31

In [66]:
# Building models using tf.keras 
# Baseline Model
model_1=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,128,input_length=train_text_padded.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(8,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])
model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
history_2=model_1.fit(train_text_padded,train_labels,epochs=4,validation_data=(validation_text_padded,validation_labels))

Epoch 1/4
1029/1029 [==============================] - 25s 24ms/step - loss: 0.7547 - accuracy: 0.6590 - val_loss: 0.5685 - val_accuracy: 0.7767
Epoch 2/4
1029/1029 [==============================] - 24s 23ms/step - loss: 0.4211 - accuracy: 0.8507 - val_loss: 0.5033 - val_accuracy: 0.8126
Epoch 3/4
1029/1029 [==============================] - 24s 23ms/step - loss: 0.3198 - accuracy: 0.8929 - val_loss: 0.5161 - val_accuracy: 0.8100
Epoch 4/4
1029/1029 [==============================] - 24s 23ms/step - loss: 0.2661 - accuracy: 0.9129 - val_loss: 0.5443 - val_accuracy: 0.8139


In [67]:
# Building LSTM Model
# Accuracy improves to ~85% on the validation dataset while using LSTM Based Architecture
model=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,128,input_length=train_text_padded.shape[1]),
    tf.keras.layers.LSTM(512),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
history_4=model.fit(train_text_padded,train_labels,epochs=4,validation_data=(validation_text_padded,validation_labels))

Epoch 1/4
1029/1029 [==============================] - 477s 461ms/step - loss: 0.7586 - accuracy: 0.6465 - val_loss: 0.5009 - val_accuracy: 0.8243
Epoch 2/4
1029/1029 [==============================] - 508s 494ms/step - loss: 0.4001 - accuracy: 0.8614 - val_loss: 0.4220 - val_accuracy: 0.8496
Epoch 3/4
1029/1029 [==============================] - 482s 468ms/step - loss: 0.2888 - accuracy: 0.9019 - val_loss: 0.4233 - val_accuracy: 0.8576
Epoch 4/4
1029/1029 [==============================] - 481s 467ms/step - loss: 0.2167 - accuracy: 0.9279 - val_loss: 0.4528 - val_accuracy: 0.8554


In [44]:
len(train_text_padded[0])

31

In [45]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 31, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1312768   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 2,761,219
Trainable params: 2,761,219
Non-trainable params: 0
______________________________________________

In [46]:
# Trying Universal Sentence Encoder from TensorHub
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [47]:
# train_embed = embed(train_text)
# test_embed = embed(validation_text)

In [48]:
# len(train_embed[0])

512

In [68]:
data_test

In [50]:
data_test.iloc[:,4]

0       TRENDING: New Yorkers encounter empty supermar...
1       When I couldn't find hand sanitizer at Fred Me...
2       Find out how you can protect yourself and love...
3       #Panic buying hits #NewYork City as anxious sh...
4       #toiletpaper #dunnypaper #coronavirus #coronav...
                              ...                        
3793    Meanwhile In A Supermarket in Israel -- People...
3794    Did you panic buy a lot of non-perishable item...
3795    Asst Prof of Economics @cconces was on @NBCPhi...
3796    Gov need to do somethings instead of biar je r...
3797    I and @ForestandPaper members are committed to...
Name: OriginalTweet, Length: 3798, dtype: object

In [51]:
for i in range(0,len(data_test)):
    if(data_test['Sentiment'][i]=='Extremely Negative'):
        data_test['Sentiment'][i]='Negative'
    elif(data_test['Sentiment'][i]=='Extremely Positive'):
        data_test['Sentiment'][i]='Positive'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [52]:
X_test=data_test.iloc[:,4]

In [53]:
##preprocessing the test dataset
for i in range(0,len(X_test)):
    temp=X_test[i]
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
    X_test[i]=temp
    ##print(temp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [54]:
final_result=[]

In [55]:
for i in range(0,len(X_test)):
    text=[X_test[i]]
    text=tokenizer.texts_to_sequences(text)
    text=pad_sequences(text,maxlen=max_length,padding='post')
    predict_result=model.predict(text)
    result=np.argmax(predict_result,axis=1)
    if(result==0):
        final_result.append('Negative')
    elif(result==1):
        final_result.append('Neutral')
    elif(result==2):
        final_result.append('Positive')
    

In [56]:
actual_result=data_test.iloc[:,5]

In [57]:
type(actual_result)

pandas.core.series.Series

In [58]:
#  Plotting the confusion matrix to check for accuracy
confusion_matrix(actual_result,final_result, labels=['Negative','Positive','Neutral'])

array([[1324,  253,   56],
       [ 130, 1380,   36],
       [  57,   88,  474]])